In [1]:
import PyPDF2

def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            text += page.extract_text() + "\n"
    return text

# Example usage
pdf_text = extract_text_from_pdf("data/papers/diet_myth.pdf")
print(pdf_text[:500])  # Print first 500 characters to verify extraction



Images
Tim Spector
THE DIET MYTH
The Real Science
Behind What We Eat
Images
Dedication
To my family and other microbes
C o n t e n t s
 Title Page
 Dedication
 Introduction: A Bad Taste
  1Not on the Label: Microbes
  2Energy and Calories
  3Fats: Total
  4Fats: Saturated
  5Fats: Unsaturated
  6Trans Fats
  7Protein: Animal
  8Protein: Non-animal
  9Protein: Milk Products
10Carbohydrates: of which Sugars
11Carbohydrates: Non-sugars
12Fibre
13Artificial Sweeteners and Preservatives
14Contains C


In [2]:
import re

def clean_and_split_text(text, chunk_size=3):
    sentences = re.split(r"(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s", text)
    chunks = [" ".join(sentences[i:i+chunk_size]) for i in range(0, len(sentences), chunk_size)]
    return chunks

# Example usage
text_chunks = clean_and_split_text(pdf_text)
print(text_chunks[:1])  # Show first 3 chunks


['\nImages\nTim Spector\nTHE DIET MYTH\nThe Real Science\nBehind What We Eat\nImages\nDedication\nTo my family and other microbes\nC o n t e n t s\n\xa0Title Page\n\xa0Dedication\n\xa0Introduction: A Bad Taste\n\xa0\xa01Not on the Label: Microbes\n\xa0\xa02Energy and Calories\n\xa0\xa03Fats: Total\n\xa0\xa04Fats: Saturated\n\xa0\xa05Fats: Unsaturated\n\xa0\xa06Trans Fats\n\xa0\xa07Protein: Animal\n\xa0\xa08Protein: Non-animal\n\xa0\xa09Protein: Milk Products\n10Carbohydrates: of which Sugars\n11Carbohydrates: Non-sugars\n12Fibre\n13Artificial Sweeteners and Preservatives\n14Contains Cocoa and Caffeine\n15Contains Alcohol\n16Vitamins\n17Warning: May Contain Antibiotics\n18Warning: May Contain Nuts\n19Best-before Date\n\xa0Conclusion: The Checkout\n\xa0\n\xa0Glossary\n\xa0Acknowledgements\n\xa0Notes\n\xa0By the same author\n\xa0Copyright\nIntroduction\nA Bad Taste\nIt had been a tough climb: six hours walking up 1,200 metres to the summit on\ntouring skis with artificial sealskins to sto

In [3]:
from transformers import TFAutoModel, AutoTokenizer
import numpy as np

# Load a Hugging Face model (BERT variant for sentence embeddings)
MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"  # Works with TensorFlow
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = TFAutoModel.from_pretrained(MODEL_NAME)




D:\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['embeddings.position_ids']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [12]:
def get_embedding(text):
    inputs = tokenizer(text, return_tensors="tf", padding=True, truncation=True)
    outputs = model(**inputs)
    return np.mean(outputs.last_hidden_state.numpy(), axis=1)  # Take mean pooling

# Convert research text chunks into embeddings
chunk_embeddings = np.array([get_embedding(chunk) for chunk in text_chunks])
shape_e=chunk_embeddings.shape
chunk_embeddings=chunk_embeddings.reshape(shape_e[0],shape_e[2])


In [21]:
def process_user_query(user_question, text_chunks, chunk_embeddings,get_embedding,threshold=0.5):
    best_match, score = find_relevant_text(user_question, text_chunks, chunk_embeddings,get_embedding)
    print(score)
    if score < threshold:
        return "Information not available in research papers."
    else:
        return best_match

# Example usage
print(process_user_query("Do artificial sweeteners cause cancer?", text_chunks, chunk_embeddings,get_embedding))


983
Chunk Embeddings Shape: (1639, 384)
Number of Text Chunks: 1639
Similarities shape: (1639,)
0.8050833
This elegant series of experiments shows that artificial sweeteners are
definitely not a free lunch; they do have potentially harmful metabolic effects
that can increase weight gain and the risk of diabetes. They do this because
even so-called inert chemicals can be crucial for our microbes, which change
their function and so affect our bodies. We don’t yet know the true extent of
the risk of the sweeteners and whether everyone is susceptible, but these
microbe experiments have made sure that we and our food regulators who
approve the new ‘safe compounds’, once they pass the cancer tests, should now
take these risks more seriously.


In [30]:
## save every thing to make compute easy
import numpy as np
import os
from transformers import AutoTokenizer, TFAutoModel

# Define save directory
save_dir = "/models/nlp/"
#os.makedirs(save_dir, exist_ok=True)

# Load a Hugging Face model (BERT variant for sentence embeddings)
MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = TFAutoModel.from_pretrained(MODEL_NAME)

# Save the model and tokenizer
model.save_pretrained("models/nlp/bert_model")
tokenizer.save_pretrained("models/nlp/bert_tokenizer")

# Example embeddings (Replace with actual embeddings)
embeddings = chunk_embeddings  # 100 embeddings of size 384

# Save embeddings
np.save("models/nlp/embeddings.npy", embeddings)

print("Model, tokenizer, and embeddings saved successfully!")


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['embeddings.position_ids']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model, tokenizer, and embeddings saved successfully!


In [39]:
np.save("models/nlp/text_chunks.npy",text_chunks)

In [33]:
from transformers import AutoTokenizer, TFAutoModel

# Load model and tokenizer
model = TFAutoModel.from_pretrained("models/nlp/bert_model")
tokenizer = AutoTokenizer.from_pretrained("models/nlp/bert_tokenizer")

# Load embeddings
chunk_embeddings = np.load("models/nlp/embeddings.npy")


print(f"Model, tokenizer, and embeddings loaded successfully! Embeddings shape: {embeddings.shape}")


All model checkpoint layers were used when initializing TFBertModel.

All the layers of TFBertModel were initialized from the model checkpoint at models/nlp/bert_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model, tokenizer, and embeddings loaded successfully! Embeddings shape: (1639, 384)


In [35]:
import numpy as np
from numpy.linalg import norm

def find_relevant_text(user_question, text_chunks, chunk_embeddings, get_embedding):
    # Convert user question into an embedding
    question_embedding = get_embedding(user_question).flatten()

    # Compute cosine similarity
    similarities = np.dot(chunk_embeddings, question_embedding)

    # Ensure similarity shape matches the number of text chunks
    assert similarities.shape[0] == len(text_chunks), "Mismatch in similarity scores and text chunks!"

    # Normalize (avoid division by zero)
    chunk_norms = norm(chunk_embeddings, axis=1)
    question_norm = norm(question_embedding)
    similarities = similarities / (chunk_norms * question_norm + 1e-8)

    # Print shapes for debugging
    print(f"Similarities shape: {similarities.shape}")
    print(f"Chunk Embeddings Shape: {chunk_embeddings.shape}")
    print(f"Number of Text Chunks: {len(text_chunks)}")

    # Get the best matching chunk
    best_match_index = np.argmax(similarities)

    # Ensure index is within bounds
    if best_match_index >= len(text_chunks):
        raise IndexError(f"Index {best_match_index} is out of bounds! Check text chunk and embedding sizes.")

    best_match_score = similarities[best_match_index]
    return text_chunks[best_match_index], best_match_score

# Example usage
user_question = "Does eating carbs at night cause weight gain?"
best_match, score = find_relevant_text(user_question, text_chunks, chunk_embeddings, get_embedding)
print(f"Best Match (Score: {score}):\n{best_match}")


Similarities shape: (1639,)
Chunk Embeddings Shape: (1639, 384)
Number of Text Chunks: 1639
Best Match (Score: 0.4809572696685791):
These mechanisms are complex, and regardless of
whether they are due to gut or brain chemicals or psychological factors (as we
learned in the introductory story of twins on diets) they are strongly influenced
by genes as well as by microbes. It is hard to distinguish in high-protein diets between how much the benefits
and side effects can be put down to the lack of carbs and how much to the
extra protein. During the recent evolution of the Atkins Diet the high priests
of the billion-dollar corporation behind it have increasingly stressed the
importance of low rather than zero carbs and reduced the emphasis on heavy
meat eating.
